## Analysis of EVI Trend Break Magnitudes - Exploratory data analysis

Peter R., 2024-02-20

### Intro
There are several way to carry out Extreme Gradient Boosting (XGB) models. R has packages such as "dismo" and "xgboosting". Similarly, there are several ways to do XGB with Python. You can use the Sklearn framework and within it you can use train(), XGBClassifier(), XGBRegressor(), etc.  Here I use XGBRegressor() to analyze forest EVI breaks (both negative & positive breaks).

Here is explore the new data set created wiht DuckDb which has more than 100 variables. The goal is to come up with a smaller set of variables to run with XGB.

Note that driver data have been assigned in the df by using spatio-temporal matches between breaks and remote sensing derived disturbance data. So far, I only have data for fire, harvest, insects, Hansen, & CanLaD  The driver data are mainly nulls as I was not able to match most of the EVI breaks with disturbance data. An important missing driver is likely tree windthrow or drought.

In analyze negative and positive breaks separately. I use variation inflation factors (VIF) to choose a smaller set of variables to run with XGB.



#### File and folder path settings

In [1]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

print(cwd)

C:\Users\Peter R\github\forc_trends\models\xgboost


#### Load, explore and prepare data for VIF

In [54]:
import pandas as pd
#from pandas import read_csv
import numpy as np
from numpy import nan

# Windows
#df1 = pd.read_csv(r'.\data\version1\forest_evi_breaks_positive_v1.csv', skipinitialspace=True)
#df1 = pd.read_csv(r'.\data\forest_evi_breaks_positive_v2.csv', skipinitialspace=True)
df1 = pd.read_csv(r'.\data\forest_evi_breaks_positive_v3.csv', skipinitialspace=True)
df1.head()


,pix,year,brk,brkdate25,brkdate,brkdate95,magnitude,no_brk,fire_year,harv_year,...,tave_wt_lag2,tave_wt_lag3,bffp,bffp_lag1,bffp_lag2,bffp_lag3,effp,effp_lag1,effp_lag2,effp_lag3
0,308165,2005,0,2005.348,2005.391,2006.087,672.967,NaN,NaN,NaN,...,-12.796,-6.163,139.000,153.000,148.000,147.883,271.000,267.0,266.0,265.000
1,308324,2005,0,2005.565,2005.609,2006.174,1070.553,NaN,NaN,NaN,...,-12.370,-5.720,136.282,151.000,146.000,146.000,272.130,268.0,267.0,266.000
2,308326,2005,0,2005.565,2005.609,2006.391,1048.369,NaN,NaN,NaN,...,-12.393,-5.772,136.317,151.000,146.000,146.000,272.000,268.0,267.0,265.683
3,310381,2005,0,2005.348,2005.391,2006.304,781.968,NaN,NaN,NaN,...,-12.296,-5.687,136.000,150.964,145.964,146.000,272.354,268.0,267.0,266.000
4,124272,2004,0,2004.000,2004.043,2004.130,458.737,NaN,NaN,NaN,...,-6.200,-11.500,150.427,145.000,146.000,140.000,269.000,268.0,266.0,270.573


In [55]:
# dataframe dimensions
df1.describe()
#print(df1.describe)
# show all column names
#print(df1.columns.tolist())

,pix,year,brk,brkdate25,brkdate,brkdate95,magnitude,no_brk,fire_year,harv_year,...,tave_wt_lag2,tave_wt_lag3,bffp,bffp_lag1,bffp_lag2,bffp_lag3,effp,effp_lag1,effp_lag2,effp_lag3
count,3935.000000,3935.000000,3935.0,3935.000000,3935.000000,3935.000000,3935.000000,0.0,0.0,102.000000,...,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000
mean,236681.719187,2012.323761,0.0,2012.462310,2012.781264,2013.527402,802.658876,NaN,NaN,2010.009804,...,-10.130569,-9.737617,141.954761,142.846732,144.181222,142.638365,269.959652,267.854668,268.369603,267.944833
std,143456.099575,4.882908,0.0,4.700422,4.795015,4.587775,390.213785,NaN,NaN,4.390573,...,1.722720,1.765085,4.829545,4.051770,4.797728,4.022379,5.123441,3.917180,4.344781,4.102968
min,601.000000,2003.000000,0.0,2003.739000,2003.957000,2004.000000,81.806000,NaN,NaN,2004.000000,...,-14.372000,-14.299000,129.248000,129.173000,129.168000,129.173000,261.000000,261.000000,260.470000,260.994000
25%,118067.000000,2009.000000,0.0,2009.217000,2009.348000,2010.826000,535.356000,NaN,NaN,2006.000000,...,-11.317000,-10.900000,138.203000,140.000000,140.819000,140.000000,265.180000,264.000000,264.911000,264.754500
50%,225129.000000,2011.000000,0.0,2011.304000,2011.652000,2012.435000,724.616000,NaN,NaN,2010.500000,...,-10.241000,-9.555000,141.000000,142.897000,143.000000,142.000000,269.780000,268.000000,268.000000,267.310000
75%,356961.500000,2016.000000,0.0,2015.739000,2016.130000,2016.478000,969.261500,NaN,NaN,2013.000000,...,-9.100000,-8.756500,145.000000,146.000000,148.875500,145.000000,273.531000,271.000000,272.000000,271.000000
max,493282.000000,2021.000000,0.0,2021.652000,2021.957000,2022.957000,3606.961000,NaN,NaN,2019.000000,...,-4.429000,-4.428000,155.000000,155.000000,155.000000,155.000000,285.000000,280.165000,281.000000,281.000000


In [29]:
#df1["rad_sm"].describe() # remove rad vars given the -9999 nffd_wt
#df1["nffd_wt"].describe() # remove rad vars given the small range 0 to 2.

In [56]:
# dataframe dimensions
df1.shape

(3935, 157)

In [57]:
# Subset the data to run VIF function below

# create non-protected dummy variable 
#X1 = pd.get_dummies(df1, columns=['for_pro'], dtype=float)

# Remove records where precipitation is null. To check why map vals have NAs
#df2= df2[df2['map'].notna()]

#X1 = X1.iloc[:,11:]
X1 = df1

# Drop response variables, driver matched data, and categorical data as these arenot useful for VIF
X1.drop(X1.columns[[0,1,2,3,4, 5, 6, 7, 8, 9,10, 11, 12, 15, 16]], axis=1, inplace=True)

#These variables have way too many NULLS, so I need to drop them. The nulls in these variables are -9999. By dropping them I solved the problem of potentila bias with -9999 values
X1.drop(["rad_sm",  "rad_sm_lag1",  "rad_sm_lag2", "rad_sm_lag3","rad_wt", "rad_wt_lag1", "rad_wt_lag2", "rad_wt_lag3"], axis = 1, inplace = True)

#replace -9999.0000
#X1.replace(-9999.0, nan, inplace=True)

# drop rows with NAs
X1.tail()
# 5 rows × 143 columns



,for_age,for_con,cmd,cmd_lag1,cmd_lag2,cmd_lag3,cmi,cmi_lag1,cmi_lag2,cmi_lag3,...,tave_wt_lag2,tave_wt_lag3,bffp,bffp_lag1,bffp_lag2,bffp_lag3,effp,effp_lag1,effp_lag2,effp_lag3
3930,66.368,0.942,161.563,135.743,114.988,162.371,42.250,58.782,68.837,52.986,...,-11.081,-10.652,137.0,146.27,150.447,147.447,277.730,263.000,264.259,267.259
3931,71.400,4.995,151.773,131.024,103.258,172.152,44.746,60.729,71.795,54.076,...,-11.100,-10.700,138.0,147.00,151.000,148.000,277.641,263.000,264.000,267.000
3932,93.352,18.592,227.234,147.678,174.031,112.024,39.376,53.872,49.332,62.020,...,-8.300,-10.000,135.0,139.00,138.000,146.000,273.000,275.406,271.000,268.000
3933,66.811,0.595,163.161,131.033,109.694,174.741,41.308,58.600,68.855,52.386,...,-11.800,-11.200,139.0,149.00,153.000,149.953,275.000,261.000,262.581,266.000
3934,89.369,3.758,160.240,26.309,221.054,135.407,62.260,81.296,43.739,37.514,...,-6.269,-12.900,146.0,140.00,144.000,145.000,268.966,276.025,272.000,270.000


In [41]:
#X1 = df1[['map', 'mat']]
# Count NAs per columns
#X1.isna().sum()

In [42]:
#X1.describe()

In [58]:
#X1['nffd_wt'].describe() # range: 0-2
#X1['nffd_wt'].describe() # range: 0-2
#X1['cmi_sm'].describe()

In [59]:
# dataframe dimensions
X1.shape

(3935, 134)

In [60]:
# Count NAs per columns
X1.isna().sum()

#X1['rad_wt_lag3'].isna().sum()

for_age      42
for_con      43
cmd           0
cmd_lag1      0
cmd_lag2      0
             ..
bffp_lag3     0
effp          0
effp_lag1     0
effp_lag2     0
effp_lag3     0
Length: 134, dtype: int64

In [61]:
# Count NAs per columns
X1.isna().sum()

for_age      42
for_con      43
cmd           0
cmd_lag1      0
cmd_lag2      0
             ..
bffp_lag3     0
effp          0
effp_lag1     0
effp_lag2     0
effp_lag3     0
Length: 134, dtype: int64

In [62]:
# drop rows with NAs (-9999.0 converted to nan) to run VIF below not for XGB model as XGB can handle NULLs
X2 = X1.dropna()
X2.shape

(3892, 134)

In [63]:
# Count NAs per columns to check that step above worked
X2.isna().sum()


for_age      0
for_con      0
cmd          0
cmd_lag1     0
cmd_lag2     0
            ..
bffp_lag3    0
effp         0
effp_lag1    0
effp_lag2    0
effp_lag3    0
Length: 134, dtype: int64

In [65]:
#X2['effp_lag3'].describe()
#s = pd.Series([-9999.0, 2, 3, 4, 5])
#s.replace(-9999.0, nan, inplace=True)
#s.describe()


In [72]:
#print(X2.columns.tolist())

In [109]:
# drop dummy vars
#X3 = X2.drop(['magnitude', 'for_pro_0',  'for_pro_1'], inplace = True)
#X2.drop(X2.columns[[0,136,137]], axis=1, inplace=True)
X2.shape

(3892, 134)

In [67]:
# remove the bad seasonal vars nffd_wt, pas_sm but keep the others. As VIF fails when there are too many nulls
cols1 =['for_age', 'for_con', 'cmd', 'cmd_lag1', 'cmd_lag2', 'cmd_lag3', 'cmi', 'cmi_lag1', 'cmi_lag2', 'cmi_lag3', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'cmi_wt', 'cmi_wt_lag1', 'cmi_wt_lag2', 'cmi_wt_lag3', 'dd1040', 'dd1040_lag1', 'dd1040_lag2', 'dd1040_lag3', 'dd18', 'dd18_lag1', 'dd18_lag2', 'dd18_lag3', 'dd18_sm', 'dd18_sm_lag1', 'dd18_sm_lag2', 'dd18_sm_lag3', 'dd5', 'dd5_lag1', 'dd5_lag2', 'dd5_lag3', 'dd5_wt', 'dd5_wt_lag1', 'dd5_wt_lag2', 'dd5_wt_lag3', 'dd_0', 'dd_0_lag1', 'dd_0_lag2', 'dd_0_lag3', 'dd_0_wt', 'dd_0_wt_lag1', 'dd_0_wt_lag2', 'dd_0_wt_lag3', 'dd_18', 'dd_18_lag1', 'dd_18_lag2', 'dd_18_lag3', 'dd_18_sm', 'dd_18_sm_lag1', 'dd_18_sm_lag2', 'dd_18_sm_lag3', 'eref', 'eref_lag1', 'eref_lag2', 'eref_lag3', 'ffp', 'ffp_lag1', 'ffp_lag2', 'ffp_lag3', 'map', 'map_lag1', 'map_lag2', 'map_lag3', 'mat', 'mat_lag1', 'mat_lag2', 'mat_lag3', 'msp', 'msp_lag1', 'msp_lag2', 'msp_lag3', 'nffd', 'nffd_lag1', 'nffd_lag2', 'nffd_lag3', 'pas', 'pas_lag1', 'pas_lag2', 'pas_lag3', 'pas_wt', 'pas_wt_lag1', 'pas_wt_lag2', 'pas_wt_lag3', 'ppt_sm', 'ppt_sm_lag1', 'ppt_sm_lag2', 'ppt_sm_lag3', 'ppt_wt', 'ppt_wt_lag1', 'ppt_wt_lag2', 'ppt_wt_lag3', 'rh', 'rh_lag1', 'rh_lag2', 'rh_lag3', 'rh_sm', 'rh_sm_lag1', 'rh_sm_lag2', 'rh_sm_lag3', 'rh_wt', 'rh_wt_lag1', 'rh_wt_lag2', 'rh_wt_lag3', 'td', 'td_lag1', 'td_lag2', 'td_lag3', 'tave_sm', 'tave_sm_lag1', 'tave_sm_lag2', 'tave_sm_lag3', 'tave_wt', 'tave_wt_lag1', 'tave_wt_lag2', 'tave_wt_lag3', 'bffp', 'bffp_lag1', 'bffp_lag2', 'bffp_lag3', 'effp', 'effp_lag1', 'effp_lag2', 'effp_lag3']
X22 = X1[cols1]

X22 = X22.dropna()


In [68]:
print(X22.describe)

#[3328 rows x 135 columns]

<bound method NDFrame.describe of       for_age  for_con      cmd  cmd_lag1  cmd_lag2  cmd_lag3     cmi  \
0      82.398   31.434  248.235   160.040   125.813   163.157  28.249   
1      83.758   12.763  263.867   169.075   132.941   199.648  22.262   
2      90.001   20.251  262.199   167.381   130.623   198.598  22.662   
3      91.311   19.127  264.709   169.716   134.123   200.587  22.037   
4      68.810   62.192  213.060   133.145   219.802   260.458  25.514   
...       ...      ...      ...       ...       ...       ...     ...   
3930   66.368    0.942  161.563   135.743   114.988   162.371  42.250   
3931   71.400    4.995  151.773   131.024   103.258   172.152  44.746   
3932   93.352   18.592  227.234   147.678   174.031   112.024  39.376   
3933   66.811    0.595  163.161   131.033   109.694   174.741  41.308   
3934   89.369    3.758  160.240    26.309   221.054   135.407  62.260   

      cmi_lag1  cmi_lag2  cmi_lag3  ...  tave_wt_lag2  tave_wt_lag3     bffp  \
0       3

In [24]:
# Caluclate Varition inflation factors
# https://datascience.stackexchange.com/questions/54280/multicollinearityvariance-inflation-factor-variables-to-remove-before-doing-a
# Note VIF (variance_inflation_factor)
from joblib import Parallel, delayed
from statsmodels.stats.outliers_influence import variance_inflation_factor
    
def removeMultiColl(data, vif_threshold = 5.0):
        for i in data.columns:
            if data[i].nunique() == 1:
                print(f"Dropping {i} due to just 1 unique value")
                data.drop(columns = i, inplace = True)
        drop = True
        col_list = list(data.columns)
        while drop == True:
            drop = False
            vif_list = Parallel(n_jobs = -1, verbose = 5)(delayed(variance_inflation_factor)(data[col_list].values, i) for i in range(data[col_list].shape[1]))
            max_index = vif_list.index(max(vif_list))
            if vif_list[max_index] > vif_threshold:
                print(f"Dropping column : {col_list[max_index]} at index - {max_index}")
                del col_list[max_index]
                drop = True
        print("Remaining columns :\n", list(data[col_list].columns))
        return data[col_list]

In [70]:
# run VIF function 
removeMultiColl(X2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 134 out of 134 | elapsed:    2.8s finished


Dropping column : dd_18_lag3 at index - 49


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 122 out of 133 | elapsed:    2.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 133 out of 133 | elapsed:    2.7s finished


Dropping column : dd_18_lag1 at index - 47


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 132 out of 132 | elapsed:    2.6s finished


...

Dropping column : dd18_sm_lag2 at index - 10
Dropping column : pas at index - 18
Dropping column : dd18_sm at index - 8
Dropping column : cmd_lag2 at index - 2


[Parallel(n_jobs=-1)]: Done  17 out of  23 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  22 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  21 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elap

Dropping column : cmd_lag3 at index - 2
Dropping column : dd18_sm_lag1 at index - 6
Dropping column : for_age at index - 0
Dropping column : dd5_wt_lag2 at index - 7
Dropping column : dd5_wt_lag1 at index - 6
Dropping column : dd5_wt_lag3 at index - 6
Remaining columns :
 ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt', 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3']


[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  17 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  17 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  16 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Us

,for_con,cmi_sm,cmi_sm_lag1,cmi_sm_lag2,cmi_sm_lag3,dd5_wt,nffd_wt,nffd_wt_lag1,nffd_wt_lag2,nffd_wt_lag3,pas_sm,pas_sm_lag1,pas_sm_lag2,pas_sm_lag3
0,31.434,-10.600,-2.034,-4.475,-5.782,5.0,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0
1,12.763,-12.612,-3.725,-4.590,-6.097,5.0,0.0,0.0,0.000,1.0,0.0,0.0,0.0,0.0
2,20.251,-12.468,-3.589,-4.417,-5.943,5.0,0.0,0.0,0.000,1.0,0.0,0.0,0.0,0.0
3,19.127,-12.661,-3.770,-4.666,-6.141,5.0,0.0,0.0,0.000,1.0,0.0,0.0,0.0,0.0
4,62.192,-5.643,-4.704,-9.364,-9.939,6.0,0.0,0.0,1.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,0.942,1.339,5.510,-3.593,-0.021,6.0,1.0,1.0,0.000,0.0,0.0,0.0,0.0,0.0
3931,4.995,3.185,5.716,-2.304,0.119,6.0,1.0,1.0,0.000,0.0,0.0,0.0,1.0,0.0
3932,18.592,-7.635,-6.538,6.704,0.837,11.0,1.0,0.0,1.000,0.0,0.0,0.0,0.0,0.0
3933,0.595,2.003,5.517,-3.183,-0.466,5.0,1.0,1.0,0.000,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Vars kept
col1 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt', 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3']

In [69]:
# this excludes the faulty seasonal vars
removeMultiColl(X22)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 126 out of 126 | elapsed:    4.4s finished


Dropping column : dd_18_lag3 at index - 49


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 114 out of 125 | elapsed:    2.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:    2.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Dropping column : dd_18_lag1 at index - 47


[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 124 out of 124 | elapsed:    2.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


...

Dropping column : tave_wt_lag3 at index - 23
Dropping column : tave_wt_lag1 at index - 22
Dropping column : cmd_lag1 at index - 2
Dropping column : dd_0_wt at index - 16


[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBac

Dropping column : pas_lag3 at index - 18
Dropping column : dd18_sm_lag3 at index - 11
Dropping column : cmd_lag2 at index - 2
Dropping column : pas_wt_lag2 at index - 16
Dropping column : pas_lag1 at index - 15
Dropping column : cmd_lag3 at index - 2
Dropping column : pas at index - 13
Dropping column : dd18_sm_lag2 at index - 8
Dropping column : dd18_sm at index - 6
Dropping column : dd18_sm_lag1 at index - 6
Dropping column : for_age at index - 0
Dropping column : dd5_wt_lag2 at index - 7
Dropping column : dd5_wt at index - 5
Remaining columns :
 ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']


[Parallel(n_jobs=-1)]: Done   6 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    0

,for_con,cmi_sm,cmi_sm_lag1,cmi_sm_lag2,cmi_sm_lag3,dd5_wt_lag1,dd5_wt_lag3
0,31.434,-10.600,-2.034,-4.475,-5.782,6.000,8.0
1,12.763,-12.612,-3.725,-4.590,-6.097,6.130,9.0
2,20.251,-12.468,-3.589,-4.417,-5.943,6.000,9.0
3,19.127,-12.661,-3.770,-4.666,-6.141,6.666,9.0
4,62.192,-5.643,-4.704,-9.364,-9.939,4.000,3.0
...,...,...,...,...,...,...,...
3930,0.942,1.339,5.510,-3.593,-0.021,6.000,5.0
3931,4.995,3.185,5.716,-2.304,0.119,6.000,5.0
3932,18.592,-7.635,-6.538,6.704,0.837,7.000,6.0
3933,0.595,2.003,5.517,-3.183,-0.466,5.000,4.0


In [81]:
# Vars kept
# with v3.csv file
cols1 =  ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']

In [77]:
# These are the vars used in the first run of XGBoost models (not including for_pro_0 as it is categorical and skewed)
cols1 =['for_age', 'for_con', 'map', 'map_lag1', 'map_lag2', 'map_lag3', 'mat', 'mat_lag1', 'mat_lag2', 'mat_lag3', 'rh', 'rh_lag1', 'rh_lag2', 'rh_lag3']
X23 = X1[cols1]

X23 = X23.dropna()

In [76]:
#print(X23.describe)

In [78]:
# this excludes the faulty seasonal vars
removeMultiColl(X23)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.0s r

Dropping column : rh_lag1 at index - 11
Dropping column : rh_lag2 at index - 11
Dropping column : rh_lag3 at index - 11
Dropping column : rh at index - 10
Dropping column : map_lag3 at index - 5
Dropping column : map_lag2 at index - 4
Dropping column : map at index - 2
Dropping column : map_lag1 at index - 2
Dropping column : mat_lag1 at index - 3
Dropping column : mat_lag3 at index - 4
Dropping column : for_age at index - 0
Dropping column : mat_lag2 at index - 2
Remaining columns :
 ['for_con', 'mat']


[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent w

,for_con,mat
0,31.434,4.412
1,12.763,4.870
2,20.251,4.821
3,19.127,4.913
4,62.192,3.857
...,...,...
3930,0.942,5.926
3931,4.995,5.800
3932,18.592,6.915
3933,0.595,5.401


In [ ]:
# Vars kept
# Origianl vars kept after VIF. Only two kept meaning my model could be simpler but non-informative.
cols1= ['for_con', 'mat']

In [79]:
# MJF asked to run a new VIF with old vars (except elevation and conifer instead od deciduos) and the VIF subset (no for_pro_0 as it is categorical and skewed or problematic vars - with too many nulls or questionable range)
cols1 =['for_age', 'for_con', 'map', 'map_lag1', 'map_lag2', 'map_lag3', 'mat', 'mat_lag1', 'mat_lag2', 'mat_lag3', 'rh', 'rh_lag1', 'rh_lag2', 'rh_lag3', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']
X24 = X1[cols1]

X24 = X24.dropna()

In [80]:
# this excludes the faulty seasonal vars
removeMultiColl(X24)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  20 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  16 out of  19 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  18 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  18 out of 

Dropping column : rh_lag2 at index - 12
Dropping column : rh_lag1 at index - 11
Dropping column : rh_lag3 at index - 11
Dropping column : rh at index - 10
Dropping column : map at index - 2
Dropping column : map_lag1 at index - 2
Dropping column : map_lag2 at index - 2
Dropping column : map_lag3 at index - 2
Dropping column : mat_lag3 at index - 5
Dropping column : mat_lag1 at index - 3
Dropping column : mat_lag2 at index - 3
Dropping column : mat at index - 2
Dropping column : for_age at index - 0
Remaining columns :
 ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']


[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  14 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  11 out of  13 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0

,for_con,cmi_sm,cmi_sm_lag1,cmi_sm_lag2,cmi_sm_lag3,dd5_wt_lag1,dd5_wt_lag3
0,31.434,-10.600,-2.034,-4.475,-5.782,6.000,8.0
1,12.763,-12.612,-3.725,-4.590,-6.097,6.130,9.0
2,20.251,-12.468,-3.589,-4.417,-5.943,6.000,9.0
3,19.127,-12.661,-3.770,-4.666,-6.141,6.666,9.0
4,62.192,-5.643,-4.704,-9.364,-9.939,4.000,3.0
...,...,...,...,...,...,...,...
3930,0.942,1.339,5.510,-3.593,-0.021,6.000,5.0
3931,4.995,3.185,5.716,-2.304,0.119,6.000,5.0
3932,18.592,-7.635,-6.538,6.704,0.837,7.000,6.0
3933,0.595,2.003,5.517,-3.183,-0.466,5.000,4.0


In [ ]:
# MJF's request, X24
# There are the variables kept, which are the same as those of X2 which was run with all the universe of variables except for_pro and problematic vars.
cols1 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']

In [32]:
import numpy as np

# Vars kept by VIF + for_age manually added for correlation plot below
cols1 = ['for_age','for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt', 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3']
X3 = X1[cols1]

print(X3.describe)


<bound method NDFrame.describe of       for_con  cmi_sm  cmi_sm_lag1  cmi_sm_lag2  cmi_sm_lag3  dd5_wt  nffd_wt  \
0      60.374  -5.753        3.632       -2.837      -11.127     5.0      0.0   
1      25.777  -4.637       10.508       -0.854       -4.958     5.0      0.0   
2      53.178  -4.739        9.859       -1.417       -5.528     5.0      0.0   
3      11.073  -3.405       11.312       -1.189       -4.410     6.0      0.0   
4      32.724  -5.511        0.523       -3.657      -14.041     5.0      0.0   
...       ...     ...          ...          ...          ...     ...      ...   
3930    2.917   5.843        6.397       -0.149        1.227     7.0      1.0   
3931   19.991  14.432        0.723       -1.454       -5.521     2.0      0.0   
3932    0.000     NaN          NaN        4.129        7.226     NaN      NaN   
3933    8.311   7.803       -0.564        2.738       11.140     7.0      1.0   
3934   19.864     NaN          NaN        5.312        4.162     NaN      N

In [63]:
# create correlation map
import matplotlib.pyplot as plt
import seaborn as sns

# Define function    
def correlation_heatmap(X):
    correlations = X.corr()

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}
                )
    

In [62]:
# Create correlation plot
#features_names1 = ["age","deciduous","elevation","precipitation","temperature","precipitation_lag1", "temperature_lag1", "precipitation_lag2", "temperature_lag2", "precipitation_lag3", "temperature_lag3","protected_0","protected_1",
#                 "cmi", "rh" ,"cmi_lag1", "rh_lag1","cmi_lag2", "rh_lag2","cmi_lag3", "rh_lag3"]

#features_names1 = ["age","deciduous","elevation","precipitation","temperature","precipitation_lag1", "temperature_lag1", "precipitation_lag2", "temperature_lag2", "precipitation_lag3", "temperature_lag3",
#                 "rh" ,"rh_lag1","rh_lag2","rh_lag3"]

#correlation_heatmap(X1[features_names1])
#correlation_heatmap(X3)
# sorted_idx = xgb.feature_importances_.argsort()

#plt.savefig('correlation5.png',  dpi=300, bbox_inches='tight')


#plt.show()

#### Create variable list for XGB models

I decided to use _v3.csv files as the match rationale between breaks and climate variables seems more robust. These files have less null records than previous versions excpect for for_age and for_con. These vars forest age and composition were derived from a new approach using Beaudoin et al. and Hermosilla et. al data sets.

I am leaving out variables that were initially selected by VIF but whose range of values is dubious (e.g., 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3'). It seems Climate NA can not downscale these metrics in a robust way. Also, I am leaving out climate variables with too many nulls (e.g. rad which had many -9999 values).

Also, I only need to include one of for_pro_0 or for_pro_1 in my models but not both. Given that the former has more records with 1  values I will use it.

#####  Variable selection after running VIF

- Vars0: Original vars (14 vars) expect elevation and deciduous has been replaced by conifers

cols1 =['for_age', 'for_con', 'map', 'map_lag1', 'map_lag2', 'map_lag3', 'mat', 'mat_lag1', 'mat_lag2', 'mat_lag3', 'rh', 'rh_lag1', 'rh_lag2', 'rh_lag3', 'for_pro_0']

- Vars1: Using all 100+ vars, X2 df above

cols1 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt', 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3']

- Vars2: Using all 100+ vars minues problematic vars (too many nulls: rad vars; stange range: nffd, pas).  X22 df above

cols1 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']

- Vars3: Using all vars in first XGB models (In the first model I kept all vars with R <0.5). Initial number of vars 14. X23 df above

cols1= ['for_con', 'mat']

- Vars4: Vars2 from above + for_age, for_pro_0

cols1 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3', 'for_age', 'for_pro_0']

- Vars5: MJF's request. Original XGB run vars (14) + Vars2.  But at the ends the same vars as in Vars2 are left.

cols1 =  ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']

In my XGB models, I will only run Vars0, Vars2, and Vars4. I will run a standard run with all records and a subset run with only records that do not have a match with disturbance data (Hansen et al.)


In [82]:
# Df 1: All vars form original XGB model 
# done already
cols1 = ['for_age', 'for_con', 'map', 'map_lag1', 'map_lag2', 'map_lag3', 'mat', 'mat_lag1', 'mat_lag2', 'mat_lag3', 'rh', 'rh_lag1', 'rh_lag2', 'rh_lag3', 'for_pro_0']


In [89]:
# Df 2: VIF subset only
cols2 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3']


In [96]:
# Df 3: VIF subset + for_age, for_pro_0 
cols3 = ['for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt_lag1', 'dd5_wt_lag3', 'for_age', 'for_pro_0']


In [97]:
# Here I test whether the right columns are selected and the df is properly created.

df2 = pd.read_csv(r'.\data\forest_evi_breaks_positive_v3.csv', skipinitialspace=True)
#df2.head()

# create non-protected dummy variable 
df3 = pd.get_dummies(df2, columns=['for_pro'], dtype=float)
#cols2 = ['for_age','for_con', 'cmi_sm', 'cmi_sm_lag1', 'cmi_sm_lag2', 'cmi_sm_lag3', 'dd5_wt', 'nffd_wt', 'nffd_wt_lag1', 'nffd_wt_lag2', 'nffd_wt_lag3', 'pas_sm', 'pas_sm_lag1', 'pas_sm_lag2', 'pas_sm_lag3', 'for_pro_0', 'for_pro_1']

# select columns for model
X6 = df3[cols1]

print(X6.describe)
#print(X6.head())

<bound method NDFrame.describe of       for_con  cmi_sm  cmi_sm_lag1  cmi_sm_lag2  cmi_sm_lag3  dd5_wt_lag1  \
0      31.434 -10.600       -2.034       -4.475       -5.782        6.000   
1      12.763 -12.612       -3.725       -4.590       -6.097        6.130   
2      20.251 -12.468       -3.589       -4.417       -5.943        6.000   
3      19.127 -12.661       -3.770       -4.666       -6.141        6.666   
4      62.192  -5.643       -4.704       -9.364       -9.939        4.000   
...       ...     ...          ...          ...          ...          ...   
3930    0.942   1.339        5.510       -3.593       -0.021        6.000   
3931    4.995   3.185        5.716       -2.304        0.119        6.000   
3932   18.592  -7.635       -6.538        6.704        0.837        7.000   
3933    0.595   2.003        5.517       -3.183       -0.466        5.000   
3934    3.758   2.089        9.817       -4.928        1.524        8.000   

      dd5_wt_lag3  for_age  for_pro_0  
0

In [98]:
# Count NAs per columns. These NAs are correct as nan. the rad variables above had -9999 values which could have biased the results.
# Hence, the rad vars were dropped.
X6.isna().sum()


for_con        43
cmi_sm          0
cmi_sm_lag1     0
cmi_sm_lag2     0
cmi_sm_lag3     0
dd5_wt_lag1     0
dd5_wt_lag3     0
for_age        42
for_pro_0       0
dtype: int64

In [99]:
X6.shape


(3935, 9)

In [100]:
X6.describe()

,for_con,cmi_sm,cmi_sm_lag1,cmi_sm_lag2,cmi_sm_lag3,dd5_wt_lag1,dd5_wt_lag3,for_age,for_pro_0
count,3892.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3935.000000,3893.000000,3935.000000
mean,32.827358,-1.534529,-0.577318,-1.497179,-1.249436,5.735864,5.616830,84.874855,0.934943
std,24.494719,5.321250,6.149891,5.223029,5.919851,2.873347,2.540958,18.584966,0.246658
min,0.000000,-16.618000,-16.886000,-16.811000,-16.882000,2.000000,2.000000,0.500000,0.000000
25%,14.624500,-5.034000,-4.588500,-4.777500,-4.870500,4.000000,4.000000,76.015000,1.000000
50%,24.649000,-1.656000,-0.876000,-0.856000,-1.239000,5.000000,5.000000,85.046000,1.000000
75%,47.640750,2.272000,4.323500,1.578000,3.304500,6.000000,6.000000,95.194000,1.000000
max,100.000000,14.549000,12.932000,11.907000,12.850000,23.313000,23.313000,174.394000,1.000000


In [101]:
# predictor
y1 = df2.iloc[:,6]
print("Response/label data")
print(y1.head())

Response/label data
0     672.967
1    1070.553
2    1048.369
3     781.968
4     458.737
Name: magnitude, dtype: float64


In [102]:
y1.describe()

count    3935.000000
mean      802.658876
std       390.213785
min        81.806000
25%       535.356000
50%       724.616000
75%       969.261500
max      3606.961000
Name: magnitude, dtype: float64

#### References

Mostly tutorials & blogs.

https://www.youtube.com/watch?v=OQKQHNCVf5k

https://www.youtube.com/watch?v=GrJP9FLV3FE&t=2167s

https://datascience.stackexchange.com/questions/16342/unbalanced-multiclass-data-with-xgboost

https://mljar.com/blog/xgboost-save-load-python/

https://machinelearningmastery.com/xgboost-for-regression/

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

https://www.datatechnotes.com/2019/06/regression-example-with-xgbregressor-in.html

https://github.com/parrt/dtreeviz

https://stackoverflow.com/questions/37627923/how-to-get-feature-importance-in-xgboost

https://towardsdatascience.com/be-careful-when-interpreting-your-features-importance-in-xgboost-6e16132588e7

https://scikit-learn.org/stable/modules/partial_dependence.html

https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html#way-partial-dependence-with-different-models

https://mljar.com/blog/xgboost-early-stopping/

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

https://github.com/parrt/dtreeviz



